In [7]:
import pandas as pd
import numpy as np
from skfuzzy import control as ctrl
from skfuzzy import membership as mf

In [8]:
#example meals database

meals = pd.DataFrame(data = {
        'calories':[300, 650, 800, 600, 400, 550],
        'protein':[10, 25, 20, 15, 18, 30],
        'fiber':[12, 4, 2, 1, 5, 6], 
        'price':[8, 7, 12, 6, 5, 10]
        }, index = ['salad', 'chicken rice', 'burger', 'fried nuggets', 'sandwich', 'grilled fish'])

In [9]:
#input variables
calorieDiff = ctrl.Antecedent(np.arange(0, 1000, 1), 'calorieDiff')
proteinDiff = ctrl.Antecedent(np.arange(0, 50, 1), 'proteinDiff')
fiberDiff = ctrl.Antecedent(np.arange(0, 20, 1), 'fiberDiff')
# priceDiff = ctrl.Antecedent(np.arange(0, 20, 1), 'priceDiff')

#output variable
suitability = ctrl.Consequent(np.arange(0, 10, 1), 'suitability')

In [10]:
#membership functions
calorieDiff['low'] = mf.trimf(calorieDiff.universe, [0, 0, 400])
calorieDiff['mid'] = mf.trimf(calorieDiff.universe, [300, 500, 700])
calorieDiff['high'] = mf.trapmf(calorieDiff.universe, [600, 800, 1000, 1000])

proteinDiff['low'] = mf.trimf(proteinDiff.universe, [0, 0, 15])
proteinDiff['mid'] = mf.trimf(proteinDiff.universe, [10, 25, 35])
proteinDiff['high'] = mf.trapmf(proteinDiff.universe, [30, 40, 50, 50])

fiberDiff['low'] = mf.trimf(fiberDiff.universe, [0, 0, 5])
fiberDiff['mid'] = mf.trimf(fiberDiff.universe, [4, 8, 12])
fiberDiff['high'] = mf.trapmf(fiberDiff.universe, [10, 15, 20, 20])

# priceDiff['low'] = mf.trapmf(priceDiff.universe, [0, 0, 5, 7])
# priceDiff['mid'] = mf.trimf(priceDiff.universe, [5, 10, 13])
# priceDiff['high'] = mf.trapmf(priceDiff.universe, [10, 15, 20, 20])

suitability['not'] = mf.trimf(suitability.universe, [0, 0, 2])
suitability['slightly'] = mf.trimf(suitability.universe, [1, 3, 4])
suitability['kinda'] = mf.trimf(suitability.universe, [3, 5, 6])
suitability['very'] = mf.trimf(suitability.universe, [5, 7, 8])
suitability['match'] = mf.trapmf(suitability.universe, [7, 9, 10, 10])

In [11]:
#system rules

rules = [
    ctrl.Rule(calorieDiff['low'] & proteinDiff['low'] & fiberDiff['low'], (suitability['match'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['very'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['low'] & fiberDiff['high'], (suitability['very'])),

    ctrl.Rule(calorieDiff['low'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(calorieDiff['low'] & proteinDiff['high'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['low'] & proteinDiff['high'] & fiberDiff['high'], (suitability['slightly'])),

    ctrl.Rule(calorieDiff['mid'] & proteinDiff['low'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['low'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(calorieDiff['mid'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['kinda'])),

    ctrl.Rule(calorieDiff['mid'] & proteinDiff['high'] & fiberDiff['low'], (suitability['kinda'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['slightly'])),
    ctrl.Rule(calorieDiff['mid'] & proteinDiff['high'] & fiberDiff['high'], (suitability['not'])),

    ctrl.Rule(calorieDiff['high'] & proteinDiff['low'] & fiberDiff['low'], (suitability['very'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['low'] & fiberDiff['mid'], (suitability['slightly'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['low'] & fiberDiff['high'], (suitability['slightly'])),

    ctrl.Rule(calorieDiff['high'] & proteinDiff['mid'] & fiberDiff['low'], (suitability['slightly'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['mid'] & fiberDiff['mid'], (suitability['not'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['mid'] & fiberDiff['high'], (suitability['not'])),

    ctrl.Rule(calorieDiff['high'] & proteinDiff['high'] & fiberDiff['low'], (suitability['slightly'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['high'] & fiberDiff['mid'], (suitability['not'])),
    ctrl.Rule(calorieDiff['high'] & proteinDiff['high'] & fiberDiff['high'], (suitability['not'])),
]

food_ctrl = ctrl.ControlSystem(rules=rules)

food = ctrl.ControlSystemSimulation(control_system=food_ctrl)

In [12]:
#user input how many nutrient they want
calories = int(input("Enter desired calories (e.g., 300-800): "))
protein = int(input("Enter desired protein in grams (e.g., 10-30): "))
fiber = int(input("Enter desired fiber in grams (e.g., 1-12): "))
budget = float(input("Enter your budget (e.g., 5-15): "))

# Display user's nutritional requirements
print("\n" + "="*50)
print("YOUR NUTRITIONAL REQUIREMENTS")
print("="*50)
print(f"Desired Calories: {calories} cal")
print(f"Desired Protein:  {protein} g")
print(f"Desired Fiber:    {fiber} g")
print(f"Budget:           ${budget}")
print("="*50)

# Function to convert numeric suitability to linguistic label
def get_suitability_label(score):
    if score < 2.5:
        return 'not'
    elif score < 4.0:
        return 'slightly'
    elif score < 5.5:
        return 'kinda'
    elif score < 7.5:
        return 'very'
    else:
        return 'match'

result = []
result.clear()

#calculate the difference in nutrients to get suitability
for meal in meals.itertuples():
    food.input['calorieDiff'] = abs(calories - meal.calories)
    food.input['proteinDiff'] = abs(protein - meal.protein)
    food.input['fiberDiff'] = abs(fiber - meal.fiber)

    food.compute()
    suit_score = food.output['suitability']
    suit_label = get_suitability_label(suit_score)
    result.append([meal.Index, suit_score, suit_label, meal.price])

# Create DataFrame with sorted results
df = pd.DataFrame(result, columns=['meal', 'suitability', 'label', 'price'])
df = df.sort_values('suitability', ascending=False)

# Filter by budget
df_in_budget = df[df['price'] <= budget]

print("\n=== All Meals (sorted by suitability) ===")
print(df.to_string(index=False))

print(f"\n=== Meals within budget (${budget}) ===")
if len(df_in_budget) > 0:
    print(df_in_budget.to_string(index=False))
else:
    print("No meals found within budget!")



YOUR NUTRITIONAL REQUIREMENTS
Desired Calories: 1000 cal
Desired Protein:  20 g
Desired Fiber:    5 g
Budget:           $20.0

=== All Meals (sorted by suitability) ===
         meal  suitability    label  price
       burger     8.183333    match     12
 chicken rice     6.890523     very      7
     sandwich     6.611111     very      5
 grilled fish     6.577778     very     10
fried nuggets     6.548148     very      6
        salad     2.577778 slightly      8

=== Meals within budget ($20.0) ===
         meal  suitability    label  price
       burger     8.183333    match     12
 chicken rice     6.890523     very      7
     sandwich     6.611111     very      5
 grilled fish     6.577778     very     10
fried nuggets     6.548148     very      6
        salad     2.577778 slightly      8
